<a href="https://colab.research.google.com/github/Run-aivle/study/blob/main/March/03.28/SEN/Sequential(LSTM%2C_GRU).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install -U finance-datareader

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import FinanceDataReader as fdr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow.keras as tf
from tensorflow import keras
from sklearn.metrics import mean_squared_error
from tensorflow.keras.utils import plot_model

In [ ]:
# 삼성전자 종목
data = fdr.DataReader('005930', '2018-01-08', '2023-03-24')
data.head()

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2018-01-08,52400,52520,51500,52020,167673,-0.001919
2018-01-09,51460,51720,49980,50400,360272,-0.031142
2018-01-10,50500,50520,48640,48840,371336,-0.030952
2018-01-11,48200,49260,48020,48240,502476,-0.012285
2018-01-12,48240,48480,46760,48200,545409,-0.000829


In [ ]:
data.tail()

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2023-03-20,61100,61200,60200,60200,9618009,-0.017945
2023-03-21,60500,60700,60100,60300,8318514,0.001661
2023-03-22,61000,61200,60500,61100,8978591,0.013267
2023-03-23,60600,62300,60600,62300,15381057,0.019640
2023-03-24,62700,63300,62300,63000,18278602,0.011236


# 데이터 전처리

In [ ]:
data.reset_index(inplace = True)
data.head()

,Date,Open,High,Low,Close,Volume,Change
0,2018-01-08,52400,52520,51500,52020,167673,-0.001919
1,2018-01-09,51460,51720,49980,50400,360272,-0.031142
2,2018-01-10,50500,50520,48640,48840,371336,-0.030952
3,2018-01-11,48200,49260,48020,48240,502476,-0.012285
4,2018-01-12,48240,48480,46760,48200,545409,-0.000829


In [ ]:
data.drop('Date', axis = 1, inplace = True)
data.head()

,Open,High,Low,Close,Volume,Change
0,52400,52520,51500,52020,167673,-0.001919
1,51460,51720,49980,50400,360272,-0.031142
2,50500,50520,48640,48840,371336,-0.030952
3,48200,49260,48020,48240,502476,-0.012285
4,48240,48480,46760,48200,545409,-0.000829


In [ ]:
data.tail()

,Open,High,Low,Close,Volume,Change
1280,61100,61200,60200,60200,9618009,-0.017945
1281,60500,60700,60100,60300,8318514,0.001661
1282,61000,61200,60500,61100,8978591,0.013267
1283,60600,62300,60600,62300,15381057,0.019640
1284,62700,63300,62300,63000,18278602,0.011236


In [ ]:
data= data.values
data.shape

(1285, 6)

In [ ]:
# 종가를 2주 단위로 보기
timestep = 10
x = np.array([data[i : i+timestep] for i in range(len(data) - timestep)])
y = np.array([data[i+timestep, -3] for i in range(len(data) - timestep)])

# test는 마지막 한주, val는 test를 제외한 마지막 1주
x_train, x_val, x_test = x[:-15], x[-15:-5], x[-5:]
y_train, y_val, y_test = y[:-15], y[-15:-5], y[-5:]

print(x.shape, y.shape)
print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)
print(x_test.shape, y_test.shape)


(1275, 10, 6) (1275,)
(1260, 10, 6) (1260,)
(10, 10, 6) (10,)
(5, 10, 6) (5,)


# 모델 학습

In [ ]:
from tensorflow.keras.backend import clear_session
from tensorflow.keras.layers import Dense, Input, Flatten
from tensorflow.keras.layers import LSTM, GRU

from tensorflow.keras.models import Model
from sklearn.metrics import mean_squared_error as MSE

In [ ]:
# 1. 세션 클리어
clear_session()

# 2. model layer 엮기
il = Input(shape = (10, 6))
hl1 = LSTM(64, return_sequences=True)(il)
hl2 = GRU(40, return_sequences=True)(hl1)
hl3 = Flatten()(hl2)
hl4 = Dense(128, activation = 'relu')(hl3)
# 종가이므로 0 값이 나오는 것을 예방
ol = Dense(1, activation = 'relu')(hl4)

# 3. model 처음과 끝 지정
model = Model(il, ol)

# 4. model 컴파일
model.compile(loss = 'mae',
              optimizer = 'adam')

# 5. 양념
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 10, 6)]           0         
                                                                 
 lstm (LSTM)                 (None, 10, 64)            18176     
                                                                 
 gru (GRU)                   (None, 10, 40)            12720     
                                                                 
 flatten (Flatten)           (None, 400)               0         
                                                                 
 dense (Dense)               (None, 128)               51328     
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 82,353
Trainable params: 82,353
Non-trainable p

In [ ]:
model.fit(x_train, y_train,
          epochs = 30, validation_data = [x_val, y_val])

Epoch 1/30
40/40 [==============================] - 5s 29ms/step - loss: 58920.3672 - val_loss: 60074.8750
Epoch 2/30
40/40 [==============================] - 0s 12ms/step - loss: 58689.7344 - val_loss: 59751.9453
Epoch 3/30
40/40 [==============================] - 0s 12ms/step - loss: 58280.7734 - val_loss: 59246.0312
Epoch 4/30
40/40 [==============================] - 0s 12ms/step - loss: 57682.6680 - val_loss: 58547.7578
Epoch 5/30
40/40 [==============================] - 0s 12ms/step - loss: 56892.2969 - val_loss: 57657.5430
Epoch 6/30
40/40 [==============================] - 0s 12ms/step - loss: 55910.2344 - val_loss: 56575.8516
Epoch 7/30
40/40 [==============================] - 0s 12ms/step - loss: 54736.9727 - val_loss: 55303.3867
Epoch 8/30
40/40 [==============================] - 1s 14ms/step - loss: 53373.4492 - val_loss: 53841.3555
Epoch 9/30
40/40 [==============================] - 1s 16ms/step - loss: 51821.1172 - val_loss: 52191.4375
Epoch 10/30
40/40 [==================

In [ ]:
y_pred = model.predict(x_test)
MSE(y_test, y_pred)**(0.5)

1/1 [==============================] - 1s 830ms/step


6277.264906790375